In [3]:
import sys
sys.path.append('..')

import torch
from torch.nn import BCELoss, BCEWithLogitsLoss
from torch.optim import Adam
from torchvision.models import resnet18

from lib.models import SimpleClassifier, ResNet
from lib.trainer import Trainer
from lib.utils import train_parse_args, get_data_loaders
from lib.constants import HYPERPARAMETERS

In [4]:
params = HYPERPARAMETERS
params['batch_size'] = 8
params['n_epochs'] = 100
params['verbose'] = True
params['version'] = 0.17

if not params['disable_cuda'] and torch.cuda.is_available():
    params['device'] = torch.device('cuda:0')
else:
    params['device'] = torch.device('cpu')
    
for p in params:
    print(f"{p} : \t{params[p]}")

batch_size : 	8
checkpoints_dir : 	/home/sergevkim/git/FacesClassification/checkpoints
checkpoint_filename : 	
disable_cuda : 	False
imgs_dir : 	/home/sergevkim/git/FacesClassification/data/CelebaHQ
label : 	Male
labels_filename : 	/home/sergevkim/git/FacesClassification/data/list_attr_celeba.txt
logs_dir : 	/home/sergevkim/git/FacesClassification/logs
n_epochs : 	100
n_imgs : 	30000
verbose : 	True
version : 	0.17
device : 	cuda:0


In [5]:
loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = SimpleClassifier()
model.cuda()

optimizer = Adam(model.parameters(), lr=1e-4)
criterion = BCELoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

O [[0.48313871]
 [0.48377076]
 [0.4830918 ]
 [0.48275641]
 [0.48384574]
 [0.48315954]
 [0.48246104]
 [0.48320505]]
L [1. 1. 1. 1. 0. 0. 0. 1.]
0 0.375
O [[0.48468196]
 [0.48320189]
 [0.48327076]
 [0.48309168]
 [0.48352838]
 [0.48331487]
 [0.48260748]
 [0.48354319]]
L [0. 0. 1. 1. 0. 0. 0. 1.]
100 0.625
O [[0.4831602 ]
 [0.48287585]
 [0.4832319 ]
 [0.48505375]
 [0.48317176]
 [0.48304701]
 [0.4830339 ]
 [0.48260248]]
L [1. 1. 1. 0. 0. 0. 1. 0.]
200 0.5
O [[0.48339853]
 [0.48227835]
 [0.48241693]
 [0.48335451]
 [0.48306569]
 [0.48256528]
 [0.48367935]
 [0.48331764]]
L [0. 0. 1. 0. 1. 0. 0. 0.]
300 0.75
! 0.562
EPOCH 1
train 1 0 0.6683883766506431
l tensor([0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)
o tensor([[0.4825],
        [0.4819],
        [0.4826],
        [0.4833],
        [0.4837],
        [0.4836],
        [0.4834],
        [0.4829]], device='cuda:0', dtype=torch.float64,
       grad_fn=<CopyBackwards>)
train 1 100 0.8140904154877209
l tensor([1., 1., 0

KeyboardInterrupt: 

In [ ]:
loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = ResNet()
model.cuda()

optimizer = Adam(model.parameters(), lr=3e-4)
criterion = BCELoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

In [ ]:
from lib.models import get_resnet
from lib.utils import log_grad_norm


loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = get_resnet()
#model.fc.register_backward_hook(log_grad_norm)
#model = SimpleClassifier()
model.cuda()

optimizer = Adam(model.parameters(), lr=1e-4)
criterion = BCEWithLogitsLoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor


a = np.array(Image.open("data/CelebaHQ/10.png"))
print(type(a))
plt.imshow(a)
a = ToTensor()(a).cuda()
a = a[np.newaxis, ...]
print(a.shape)


print(model(a))

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from scipy.linalg import norm

a = np.array([0, 1, 1])
b = np.array([0.1, 0.2, 2.])

#b = normalize(b[..., np.newaxis])
b = np.round(b)
print(b)

accuracy_score(a, b, normalize=True)

[0. 0. 2.]


0.3333333333333333

In [ ]:
model = get_resnet()
print(model)

In [ ]:
import torch 
a = torch.ones(5)
a.requires_grad = True

b = 2*a

b.retain_grad()   # Since b is non-leaf and it's grad will be destroyed otherwise.

c = b.mean()

c.backward()

print(a.grad, b.grad)

# Redo the experiment but with a hook that multiplies b's grad by 2. 
a = torch.ones(5)

a.requires_grad = True

b = 2*a

b.retain_grad()

b.register_hook(lambda x: print('!', x))  
print(123)
b.mean().backward() 
print(456)
print(a.grad, b.grad)